In [39]:
import numpy as np
import pandas as pd
from scipy.integrate import trapezoid
from scipy.optimize import curve_fit
from scipy.stats import linregress

In [40]:
data_csv = pd.read_csv("hq_markup_train.csv")
bq_data_csv = pd.read_csv("markup_train.csv")
# data_csv

In [ ]:
bq_data_csv.isna().sum()

In [42]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
from scipy.integrate import trapezoid

def calculate_file_stats(data_csv: pd.DataFrame) -> pd.DataFrame:
    """
    Вычисляет статистические характеристики для каждого файла.

    Параметры:
    ----------
    data_csv : pandas.DataFrame
        DataFrame, содержащий столбец 'file_name' с именами файлов, которые нужно обработать.

    Возвращает:
    ----------
    stats_by_file : pandas.DataFrame
        DataFrame, содержащий статистические характеристики для каждого файла. Включает следующие столбцы:
        - file_name: имя файла
        - max_deltaP: максимальное значение deltaP
        - min_deltaP: минимальное значение deltaP
        - max_dP: максимальное значение dP
        - min_dP: минимальное значение dP
        - mean_deltaP: среднее значение deltaP
        - mean_dP: среднее значение dP
        - std_dP: стандартное отклонение dP
        - peak_time: время пика dP
        - slope_dP_before_peak: наклон dP до пика
        - slope_dP_after_peak: наклон dP после пика
        - integral_dP: интеграл dP по времени
    """
    # пустой DataFrame для хранения статистики
    stats_by_file = pd.DataFrame(columns=[
        'file_name',
        'max_deltaP',
        'min_deltaP',
        'max_dP',
        'min_dP',
        'mean_deltaP',
        'mean_dP',
        'std_dP',
        'peak_time',
        'slope_dP_before_peak',
        'slope_dP_after_peak',
        'integral_dP'
    ])

    for file_name in data_csv['file_name']:
        
        data = pd.read_csv(f'data/{file_name}', delimiter='\t', names=['t', 'deltaP', 'dP'])
        
        max_deltaP = data['deltaP'].max()
        min_deltaP = data['deltaP'].min()
        max_dP = data['dP'].max()
        min_dP = data['dP'].min()
        mean_deltaP = data['deltaP'].mean()
        mean_dP = data['dP'].mean()
        std_dP = data['dP'].std()

        peak_idx = data["dP"].idxmax()
        peak_time = data.loc[peak_idx, 't']

        slope_before, *_ = linregress(data['t'][:peak_idx+1], data["dP"][:peak_idx+1])
        slope_after, *_ = linregress(data['t'][peak_idx:], data["dP"][peak_idx:])
        slope_dP_before_peak = slope_before
        slope_dP_after_peak = slope_after

        integral_dP = trapezoid(data["dP"], data['t'])

        features_dict = {
            'file_name': file_name,
            'max_deltaP': max_deltaP,
            'min_deltaP': min_deltaP,
            'max_dP': max_dP,
            'min_dP': min_dP,
            'mean_deltaP': mean_deltaP,
            'mean_dP': mean_dP,
            'std_dP': std_dP,
            'peak_time': peak_time,
            'slope_dP_before_peak': slope_dP_before_peak,
            'slope_dP_after_peak': slope_dP_after_peak,
            'integral_dP': integral_dP
        }

        features_df = pd.DataFrame([features_dict])
        stats_by_file = pd.concat([stats_by_file, features_df], ignore_index=True)

    return stats_by_file

Удалим все строки, которые указывают на пустой файл

In [43]:
indices_to_drop = []
print(bq_data_csv.shape)

for index, file_name in enumerate(bq_data_csv['file_name']):
    txt_file = pd.read_csv(f'data/{file_name}', delimiter='\t', names=['t', 'delta_p', 'dp'])
    if len(txt_file) == 0:
        indices_to_drop.append(index)

bq_data_csv = bq_data_csv.drop(indices_to_drop).reset_index(drop=True)
bq_data_csv.shape

(45141, 18)


(44884, 18)

In [ ]:
stats_by_file = calculate_file_stats(bq_data_csv)

In [49]:
col_name = 'Билинейный режим'
stats_by_file[col_name] = bq_data_csv[col_name]

In [50]:
from sklearn.model_selection import train_test_split
X = stats_by_file.drop(columns=[col_name, 'file_name'])
y = stats_by_file[col_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=5, class_weight='balanced')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'train: {accuracy_score(y_train, model.predict(X_train))}')
print(f'test: {accuracy_score(y_test, model.predict(X_test))}')

train: 0.7045701395271117
test: 0.7051353458839256


In [52]:
model.feature_importances_

array([0.03567306, 0.00876578, 0.02798169, 0.10144522, 0.02677408,
       0.02498522, 0.04231277, 0.28681263, 0.17584086, 0.1777885 ,
       0.09162018])